In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 10.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.7 MB/s 
     |████████████████████████████████| 115 kB 58.6 MB/s 
     |████████████████████████████████| 212 kB 60.3 MB/s 
     |████████████████████████████████| 141 kB 70.9 MB/s 
     |████████████████████████████████| 127 kB 70.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully 

In [2]:
import os
import itertools
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from datasets import Dataset
from datasets import load_metric
import warnings
import spacy
from spacy.training import offsets_to_biluo_tags

In [4]:
!pip install spacy

warnings.filterwarnings('ignore')

data = pd.read_json(path_or_buf='admin.jsonl', lines=True)
cls = spacy.util.get_lang_class('en') 
nlp = cls()

tags_list = []
tokens_list = []

for i in range(len(data)):
    doc = nlp(data['text'][i])
    doc_tokens= []
    for d in range(len(doc)):
        doc_tokens.append(str(doc[d]))
    tokens_list.append(doc_tokens)
    entities = data['label'][i]
    tags = offsets_to_biluo_tags(doc, entities)
    tags_list.append(tags)

data['tokens'] = tokens_list
data['ner_tags'] = tags_list

data = data.drop(['id', 'label', 'text'], axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
train_percent = 0.8
train_size = int(train_percent*len(data))
train_df = data[:train_size]
test_df = data[train_size:]

print("FULL Dataset: {}".format(len(data)))
print("TRAIN Dataset: {}".format(len(train_df)))
print("TEST Dataset: {}".format(len(test_df)))

FULL Dataset: 500
TRAIN Dataset: 400
TEST Dataset: 100


In [8]:
label_list = ['B-code', 'I-code', 'L-code', 'U-code', 'B-address', 'I-address', 'L-address', 'U-address', 
                 'B-event', 'I-event', 'L-event', 'U-event', 'B-name', 'I-name', 'L-name', 'U-name', 'O', '-']

label_encoding_dict = {'B-code': 0, 'I-code': 1, 'L-code': 2, 'U-code': 3, 'B-address': 4, 'I-address': 5, 'L-address': 6, 
                       'U-address': 7, 'B-event': 8, 'I-event': 9, 'L-event': 10, 'U-event': 11, 'B-name': 12, 'I-name': 13,
                       'L-name': 14, 'U-name': 15, 'O': 16, '-': 17}

task = "ner" 
model_checkpoint = "bert-base-uncased"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def get_all_tokens_and_ner_tags(directory):
    return pd.concat([get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in os.listdir(directory)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['tokens', 'ner_tags'])
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['tokens', 'ner_tags'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Run model without pre-training

In [11]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True)

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"], 
            "address-precision": results["address"]["precision"], "address-recall": results["address"]["recall"], "address-f1": results["address"]["f1"], "address-number": results["address"]["number"], 
            "name-precision": results["name"]["precision"], "name-recall": results["name"]["recall"], "name-f1": results["name"]["f1"], "name-number": results["name"]["number"],  
            "event-precision": results["event"]["precision"], "event-recall": results["event"]["recall"], "event-f1": results["event"]["f1"], "event-number": results["event"]["number"], 
            "code-precision": results["code"]["precision"], "code-recall": results["code"]["recall"], "code-f1": results["code"]["f1"], "code-number": results["code"]["number"]}
 
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "int

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Address-precision,Address-recall,Address-f1,Address-number,Name-precision,Name-recall,Name-f1,Name-number,Event-precision,Event-recall,Event-f1,Event-number,Code-precision,Code-recall,Code-f1,Code-number
1,0.951300,0.411871,0.509202,0.530351,0.519562,0.892996,0.514423,0.835938,0.636905,128,0.000000,0.000000,0.000000,19,0.584270,0.530612,0.556150,98,0.241379,0.104478,0.145833,67
2,0.308600,0.211556,0.728324,0.805112,0.764795,0.943666,0.692771,0.898438,0.782313,128,0.250000,0.105263,0.148148,19,0.848485,0.857143,0.852792,98,0.698630,0.761194,0.728571,67
3,0.158600,0.171656,0.802395,0.856230,0.828439,0.957377,0.801370,0.914062,0.854015,128,0.650000,0.684211,0.666667,19,0.823529,0.857143,0.840000,98,0.818182,0.805970,0.812030,67
4,0.103400,0.163328,0.814925,0.872204,0.842593,0.958569,0.815068,0.929688,0.868613,128,0.708333,0.894737,0.790698,19,0.857143,0.857143,0.857143,98,0.791045,0.791045,0.791045,67


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


{'eval_loss': 0.16332849860191345,
 'eval_precision': 0.8149253731343283,
 'eval_recall': 0.8722044728434505,
 'eval_f1': 0.8425925925925926,
 'eval_accuracy': 0.9585692995529062,
 'eval_address-precision': 0.815068493150685,
 'eval_address-recall': 0.9296875,
 'eval_address-f1': 0.8686131386861314,
 'eval_address-number': 128,
 'eval_name-precision': 0.7083333333333334,
 'eval_name-recall': 0.8947368421052632,
 'eval_name-f1': 0.7906976744186046,
 'eval_name-number': 19,
 'eval_event-precision': 0.8571428571428571,
 'eval_event-recall': 0.8571428571428571,
 'eval_event-f1': 0.8571428571428571,
 'eval_event-number': 98,
 'eval_code-precision': 0.7910447761194029,
 'eval_code-recall': 0.7910447761194029,
 'eval_code-f1': 0.7910447761194029,
 'eval_code-number': 67,
 'eval_runtime': 0.6234,
 'eval_samples_per_second': 160.411,
 'eval_steps_per_second': 11.229,
 'epoch': 4.0}

In [12]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

def tokenize_and_align_labels_pretraining(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


inputs = tokenizer(dataset["train"][0]["tokens"], is_split_into_words=True)

tokenized_dataset= dataset.map(
    tokenize_and_align_labels_pretraining,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_names = dataset["train"].features["ner_tags"].feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

label_names = dataset["train"].features["ner_tags"].feature.names
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id)


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm

Pre-train

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model('./bert_ner_pt') #For reuse

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss
1,0.088100,0.048582
2,0.033700,0.046440
3,0.017400,0.047583


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.j

Run model with pre-training

In [25]:
model = AutoModelForTokenClassification.from_pretrained('bert_ner_pt', num_labels=len(label_list), ignore_mismatched_sizes=True)

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"], 
            "address-precision": results["address"]["precision"], "address-recall": results["address"]["recall"], "address-f1": results["address"]["f1"], "address-number": results["address"]["number"], 
            "name-precision": results["name"]["precision"], "name-recall": results["name"]["recall"], "name-f1": results["name"]["f1"], "name-number": results["name"]["number"],  
            "event-precision": results["event"]["precision"], "event-recall": results["event"]["recall"], "event-f1": results["event"]["f1"], "event-number": results["event"]["number"], 
            "code-precision": results["code"]["precision"], "code-recall": results["code"]["recall"], "code-f1": results["code"]["f1"], "code-number": results["code"]["number"]}
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

loading configuration file bert_ner_pt/config.json
Model config BertConfig {
  "_name_or_path": "bert_ner_pt",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
    

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Address-precision,Address-recall,Address-f1,Address-number,Name-precision,Name-recall,Name-f1,Name-number,Event-precision,Event-recall,Event-f1,Event-number,Code-precision,Code-recall,Code-f1,Code-number
1,0.775000,0.263884,0.796296,0.824281,0.810047,0.936811,0.790541,0.914062,0.847826,128,0.750000,0.789474,0.769231,19,0.861702,0.826531,0.843750,98,0.725806,0.671642,0.697674,67
2,0.231800,0.172860,0.853035,0.853035,0.853035,0.957079,0.808511,0.890625,0.847584,128,0.928571,0.684211,0.787879,19,0.923913,0.867347,0.894737,98,0.833333,0.820896,0.827068,67
3,0.125000,0.183569,0.814925,0.872204,0.842593,0.958271,0.800000,0.906250,0.849817,128,0.681818,0.789474,0.731707,19,0.908163,0.908163,0.908163,98,0.757143,0.791045,0.773723,67
4,0.075600,0.178419,0.837423,0.872204,0.854460,0.961252,0.815603,0.898438,0.855019,128,0.727273,0.842105,0.780488,19,0.936842,0.908163,0.922280,98,0.791045,0.791045,0.791045,67
5,0.055300,0.185633,0.835866,0.878594,0.856698,0.961550,0.821429,0.898438,0.858209,128,0.695652,0.842105,0.761905,19,0.928571,0.928571,0.928571,98,0.791045,0.791045,0.791045,67
6,0.037900,0.191005,0.831325,0.881789,0.855814,0.960358,0.811189,0.906250,0.856089,128,0.727273,0.842105,0.780488,19,0.928571,0.928571,0.928571,98,0.779412,0.791045,0.785185,67


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


{'eval_loss': 0.19100475311279297,
 'eval_precision': 0.8313253012048193,
 'eval_recall': 0.8817891373801917,
 'eval_f1': 0.8558139534883721,
 'eval_accuracy': 0.9603576751117735,
 'eval_address-precision': 0.8111888111888111,
 'eval_address-recall': 0.90625,
 'eval_address-f1': 0.8560885608856088,
 'eval_address-number': 128,
 'eval_name-precision': 0.7272727272727273,
 'eval_name-recall': 0.8421052631578947,
 'eval_name-f1': 0.7804878048780488,
 'eval_name-number': 19,
 'eval_event-precision': 0.9285714285714286,
 'eval_event-recall': 0.9285714285714286,
 'eval_event-f1': 0.9285714285714286,
 'eval_event-number': 98,
 'eval_code-precision': 0.7794117647058824,
 'eval_code-recall': 0.7910447761194029,
 'eval_code-f1': 0.7851851851851852,
 'eval_code-number': 67,
 'eval_runtime': 0.6164,
 'eval_samples_per_second': 162.234,
 'eval_steps_per_second': 11.356,
 'epoch': 6.0}